# Importando as bibliotecas básicas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importando o dataset

In [2]:
dados = pd.read_excel(r'/Users/pedro.alves/Documents/Pessoal/PA Analytics/Cursos/Python para Data Science e Analytics/Curso/V2.0/Materiais/3 - Modulo Avançado/Aula 5 - Random Forest/Data_Train.xlsx')

Este dataset tem diversos dados de voos com seu preço associado

O objetivo do problema é conseguir prever o preço de um voo através dos dados disponíveis

# Olhando para os dados

In [3]:
dados

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [4]:
dados.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [5]:
dados.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

Temos muitas colunas categóricas neste dataset, então vamos tentar quebrar algumas, de forma a construir algumas variáveis numéricas

# Tratamento dos Dados

## Quebrando a Variável de Data

In [6]:
dados['day'] = pd.Series()
dados['month'] = pd.Series()
dados['year'] = pd.Series()

from datetime import datetime


for i in dados.index:
    
    dados['day'][i] = datetime.strptime(dados.Date_of_Journey[i], '%d/%m/%Y').day
    dados['month'][i] = datetime.strptime(dados.Date_of_Journey[i], '%d/%m/%Y').month
    dados['year'][i] = datetime.strptime(dados.Date_of_Journey[i], '%d/%m/%Y').year


<ipython-input-6-0ebd57611116>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['day'] = pd.Series()
<ipython-input-6-0ebd57611116>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['month'] = pd.Series()
<ipython-input-6-0ebd57611116>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['year'] = pd.Series()
<ipython-input-6-0ebd57611116>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['day'][i] = datetime.strptime(dados.

In [7]:
dados

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day,month,year
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24.0,3.0,2019.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1.0,5.0,2019.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9.0,6.0,2019.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12.0,5.0,2019.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1.0,3.0,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107,9.0,4.0,2019.0
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145,27.0,4.0,2019.0
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229,27.0,4.0,2019.0
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648,1.0,3.0,2019.0


In [8]:
dados = dados.drop('Date_of_Journey', axis = 1)

## Quebrando a variável de Hora (Partida e Chegada)

In [9]:
dados['departure_hour'] = pd.Series()
dados['departure_min'] = pd.Series()
dados['arrival_hour'] = pd.Series()
dados['arrival_min'] = pd.Series()


for i in dados.index:
    
    dados['departure_hour'][i]= dados.Dep_Time[i][0:2]
    dados['departure_min'][i] = dados.Dep_Time[i][3:6]
    dados['arrival_hour'][i] = dados.Arrival_Time[i][0:2]
    dados['arrival_min'][i] = dados.Arrival_Time[i][3:6]


<ipython-input-9-507458aef805>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['departure_hour'] = pd.Series()
<ipython-input-9-507458aef805>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['departure_min'] = pd.Series()
<ipython-input-9-507458aef805>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['arrival_hour'] = pd.Series()
<ipython-input-9-507458aef805>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['arrival_min'] = pd.Series()
<ipython-input-9-507458aef805>:9: SettingWithC

In [10]:
dados

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day,month,year,departure_hour,departure_min,arrival_hour,arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24.0,3.0,2019.0,22.0,20.0,1.0,10.0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1.0,5.0,2019.0,5.0,50.0,13.0,15.0
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9.0,6.0,2019.0,9.0,25.0,4.0,25.0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12.0,5.0,2019.0,18.0,5.0,23.0,30.0
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1.0,3.0,2019.0,16.0,50.0,21.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107,9.0,4.0,2019.0,19.0,55.0,22.0,25.0
10679,Air India,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145,27.0,4.0,2019.0,20.0,45.0,23.0,20.0
10680,Jet Airways,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229,27.0,4.0,2019.0,8.0,20.0,11.0,20.0
10681,Vistara,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648,1.0,3.0,2019.0,11.0,30.0,14.0,10.0


In [11]:
dados = dados.drop('Dep_Time', axis = 1)
dados = dados.drop('Arrival_Time', axis = 1)

In [12]:
dados

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,day,month,year,departure_hour,departure_min,arrival_hour,arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24.0,3.0,2019.0,22.0,20.0,1.0,10.0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,1.0,5.0,2019.0,5.0,50.0,13.0,15.0
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,9.0,6.0,2019.0,9.0,25.0,4.0,25.0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,12.0,5.0,2019.0,18.0,5.0,23.0,30.0
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,1.0,3.0,2019.0,16.0,50.0,21.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,CCU → BLR,2h 30m,non-stop,No info,4107,9.0,4.0,2019.0,19.0,55.0,22.0,25.0
10679,Air India,Kolkata,Banglore,CCU → BLR,2h 35m,non-stop,No info,4145,27.0,4.0,2019.0,20.0,45.0,23.0,20.0
10680,Jet Airways,Banglore,Delhi,BLR → DEL,3h,non-stop,No info,7229,27.0,4.0,2019.0,8.0,20.0,11.0,20.0
10681,Vistara,Banglore,New Delhi,BLR → DEL,2h 40m,non-stop,No info,12648,1.0,3.0,2019.0,11.0,30.0,14.0,10.0


Agora que temos algumas variáveis novas na nossa base de dados, vamos tratar as variáveis categóricas

## Tratando a variável de quantidade de paradas

In [13]:
dados.Total_Stops.unique()


array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [14]:
dados.Total_Stops = dados.Total_Stops.fillna('non-stop')
dados['stops'] = pd.Series()

for i in dados.index:
    if dados.Total_Stops[i] == 'non-stop':
        dados.stops[i] =0
    else:
        dados.stops[i] = dados.Total_Stops[i][0:2]

<ipython-input-14-5640c32c0051>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dados['stops'] = pd.Series()
<ipython-input-14-5640c32c0051>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados.stops[i] =0
<ipython-input-14-5640c32c0051>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados.stops[i] = dados.Total_Stops[i][0:2]


In [15]:
dados

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,day,month,year,departure_hour,departure_min,arrival_hour,arrival_min,stops
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24.0,3.0,2019.0,22.0,20.0,1.0,10.0,0.0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,1.0,5.0,2019.0,5.0,50.0,13.0,15.0,2.0
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,9.0,6.0,2019.0,9.0,25.0,4.0,25.0,2.0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,12.0,5.0,2019.0,18.0,5.0,23.0,30.0,1.0
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,1.0,3.0,2019.0,16.0,50.0,21.0,35.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,CCU → BLR,2h 30m,non-stop,No info,4107,9.0,4.0,2019.0,19.0,55.0,22.0,25.0,0.0
10679,Air India,Kolkata,Banglore,CCU → BLR,2h 35m,non-stop,No info,4145,27.0,4.0,2019.0,20.0,45.0,23.0,20.0,0.0
10680,Jet Airways,Banglore,Delhi,BLR → DEL,3h,non-stop,No info,7229,27.0,4.0,2019.0,8.0,20.0,11.0,20.0,0.0
10681,Vistara,Banglore,New Delhi,BLR → DEL,2h 40m,non-stop,No info,12648,1.0,3.0,2019.0,11.0,30.0,14.0,10.0,0.0


In [16]:
dados = dados.drop('Total_Stops', axis = 1)

# Aplicando o Label Encoder

In [17]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

colunas = dados.dtypes.reset_index()

categ_cols = colunas[colunas[0] == 'object']['index'].to_list()


# Preenchendo as variáveis com dados nulos com a média


for i in categ_cols:
    dados[str(i) +'_encoded'] = le.fit_transform(dados[i])
    dados = dados.drop(i,axis = 1)


In [18]:
dados

,Price,day,month,year,departure_hour,departure_min,arrival_hour,arrival_min,stops,Airline_encoded,Source_encoded,Destination_encoded,Route_encoded,Duration_encoded,Additional_Info_encoded
0,3897,24.0,3.0,2019.0,22.0,20.0,1.0,10.0,0.0,3,0,5,18,240,8
1,7662,1.0,5.0,2019.0,5.0,50.0,13.0,15.0,2.0,1,3,0,84,336,8
2,13882,9.0,6.0,2019.0,9.0,25.0,4.0,25.0,2.0,4,2,1,118,106,8
3,6218,12.0,5.0,2019.0,18.0,5.0,23.0,30.0,1.0,3,3,0,91,311,8
4,13302,1.0,3.0,2019.0,16.0,50.0,21.0,35.0,1.0,3,0,5,29,303,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,4107,9.0,4.0,2019.0,19.0,55.0,22.0,25.0,0.0,0,3,0,64,236,8
10679,4145,27.0,4.0,2019.0,20.0,45.0,23.0,20.0,0.0,1,3,0,64,237,8
10680,7229,27.0,4.0,2019.0,8.0,20.0,11.0,20.0,0.0,4,0,2,18,280,8
10681,12648,1.0,3.0,2019.0,11.0,30.0,14.0,10.0,0.0,10,0,5,18,238,8


# Quebrando o dataset para modelagem

In [19]:
from sklearn.model_selection import train_test_split

x = dados.drop('Price', axis = 1)
y = dados['Price']

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3, random_state=42)

# Carregando o modelo

Iremos fazer o mesmo processo para os 3 modelos que falamos para verificar as diferenças

In [20]:
!pip install xgboost

You should consider upgrading via the '/Users/pedro.alves/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [21]:
!pip install lightgbm

You should consider upgrading via the '/Users/pedro.alves/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [22]:
!pip3 install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.3
    Uninstalling pip-22.0.3:
      Successfully uninstalled pip-22.0.3


In [23]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

import time

modelo_gr = GradientBoostingRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)
modelo_xb = XGBRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)
modelo_lg = LGBMRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)

print('=========== Gradient Boosting ==============')

start = time.time()
modelo_gr.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_gr.predict(x_train)
y_pred_test = modelo_gr.predict(x_test)

print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))

print('=========== XG Boost ==============')

start = time.time()
modelo_xb.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_xb.predict(x_train)
y_pred_test = modelo_xb.predict(x_test)


print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))


print('=========== LGBM ==============')

start = time.time()
modelo_lg.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_lg.predict(x_train)
y_pred_test = modelo_lg.predict(x_test)

print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))


=========== Gradient Boosting ==============
O resultado na base de treino é:  0.9348709064319582
O resultado na base de teste é:  0.8934368046644536
O tempo que o modelo demorou para treinar foi:  1.8022782802581787
=========== XG Boost ==============
[10:39:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


O resultado na base de treino é:  0.9957181114160927
O resultado na base de teste é:  0.901175584154553
O tempo que o modelo demorou para treinar foi:  3.1851937770843506
=========== LGBM ==============
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=20 will be ignored. Current value: num_leaves=31
O resultado na base de treino é:  0.9207

É possível ver as principais diferenças entre os modelos

- O GradientBoosting é um bom parâmetro para início

- O XGBoost traz resultados mais precisos

- O LGBM é treinado com uma velocidade muito maior

# Fazendo a Feature Selection

## Select KBest

In [24]:
from sklearn.feature_selection import SelectKBest, f_regression

In [25]:
selector = SelectKBest(score_func=f_regression, k = 7)

# Passamos todo o Dataframe para a Feature Selection, não apenas os dados de treino
selector.fit(x,y)

dados_selected = selector.transform(x)

/Users/pedro.alves/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/Users/pedro.alves/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:306: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom


In [26]:
dados_selected

array([[ 24.,   3.,  10., ...,   5.,  18., 240.],
       [  1.,   5.,  15., ...,   0.,  84., 336.],
       [  9.,   6.,  25., ...,   1., 118., 106.],
       ...,
       [ 27.,   4.,  20., ...,   2.,  18., 280.],
       [  1.,   3.,  10., ...,   5.,  18., 238.],
       [  9.,   5.,  15., ...,   1., 108., 347.]])

In [29]:
cols = selector.get_support(indices=True)
dados_new_best = x.iloc[:,cols]

In [30]:
dados_new_best

,day,month,arrival_min,stops,Destination_encoded,Route_encoded,Duration_encoded
0,24.0,3.0,10.0,0.0,5,18,240
1,1.0,5.0,15.0,2.0,0,84,336
2,9.0,6.0,25.0,2.0,1,118,106
3,12.0,5.0,30.0,1.0,0,91,311
4,1.0,3.0,35.0,1.0,5,29,303
...,...,...,...,...,...,...,...
10678,9.0,4.0,25.0,0.0,0,64,236
10679,27.0,4.0,20.0,0.0,0,64,237
10680,27.0,4.0,20.0,0.0,2,18,280
10681,1.0,3.0,10.0,0.0,5,18,238


Estas são as 7 colunas (features) que mais influenciam no preço das passagens

## Select Percentile

In [31]:
from sklearn.feature_selection import SelectPercentile, f_regression


selector = SelectPercentile(score_func=f_regression,percentile=50)
selector.fit(x,y)
dados_selected = selector.transform(x)


cols = selector.get_support(indices=True)
dados_new_percentile = x.iloc[:,cols]

/Users/pedro.alves/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/Users/pedro.alves/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:306: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom


In [32]:
dados_new_percentile

,day,month,arrival_min,stops,Destination_encoded,Route_encoded,Duration_encoded
0,24.0,3.0,10.0,0.0,5,18,240
1,1.0,5.0,15.0,2.0,0,84,336
2,9.0,6.0,25.0,2.0,1,118,106
3,12.0,5.0,30.0,1.0,0,91,311
4,1.0,3.0,35.0,1.0,5,29,303
...,...,...,...,...,...,...,...
10678,9.0,4.0,25.0,0.0,0,64,236
10679,27.0,4.0,20.0,0.0,0,64,237
10680,27.0,4.0,20.0,0.0,2,18,280
10681,1.0,3.0,10.0,0.0,5,18,238


# Rodando Novamente os Modelos com a Feature Selection Realizada

## Select KBest

In [33]:
x_train,x_test,y_train,y_test = train_test_split(dados_new_best,y, test_size=0.3, random_state=42)

In [34]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

import time

modelo_gr = GradientBoostingRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)
modelo_xb = XGBRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)
modelo_lg = LGBMRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)

print('=========== Gradient Boosting ==============')

start = time.time()
modelo_gr.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_gr.predict(x_train)
y_pred_test = modelo_gr.predict(x_test)

print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))

print('=========== XG Boost ==============')

start = time.time()
modelo_xb.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_xb.predict(x_train)
y_pred_test = modelo_xb.predict(x_test)


print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))


print('=========== LGBM ==============')

start = time.time()
modelo_lg.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_lg.predict(x_train)
y_pred_test = modelo_lg.predict(x_test)

print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))


=========== Gradient Boosting ==============
O resultado na base de treino é:  0.8491028168273105
O resultado na base de teste é:  0.7251419119722786
O tempo que o modelo demorou para treinar foi:  1.170379877090454
=========== XG Boost ==============
[10:45:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


O resultado na base de treino é:  0.9593375515119554
O resultado na base de teste é:  0.6864197496260348
O tempo que o modelo demorou para treinar foi:  2.5847327709198
=========== LGBM ==============
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=20 will be ignored. Current value: num_leaves=31
O resultado na base de treino é:  0.8250323

## Select Percentile

In [35]:
x_train,x_test,y_train,y_test = train_test_split(dados_new_percentile,y, test_size=0.3, random_state=42)


from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

import time

modelo_gr = GradientBoostingRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)
modelo_xb = XGBRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)
modelo_lg = LGBMRegressor(max_depth=10, max_leaf_nodes=20, random_state=42)

print('=========== Gradient Boosting ==============')

start = time.time()
modelo_gr.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_gr.predict(x_train)
y_pred_test = modelo_gr.predict(x_test)

print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))

print('=========== XG Boost ==============')

start = time.time()
modelo_xb.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_xb.predict(x_train)
y_pred_test = modelo_xb.predict(x_test)


print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))


print('=========== LGBM ==============')

start = time.time()
modelo_lg.fit(x_train, y_train)
end = time.time()
y_pred_train = modelo_lg.predict(x_train)
y_pred_test = modelo_lg.predict(x_test)

print('O resultado na base de treino é: ',r2_score(y_train,y_pred_train))
print('O resultado na base de teste é: ',r2_score(y_test,y_pred_test))
print('O tempo que o modelo demorou para treinar foi: ', str(end-start))


=========== Gradient Boosting ==============
O resultado na base de treino é:  0.8491028168273105
O resultado na base de teste é:  0.7251419119722786
O tempo que o modelo demorou para treinar foi:  1.5005171298980713
=========== XG Boost ==============
[10:45:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


O resultado na base de treino é:  0.9593375515119554
O resultado na base de teste é:  0.6864197496260348
O tempo que o modelo demorou para treinar foi:  3.396089792251587
=========== LGBM ==============
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=20 will be ignored. Current value: num_leaves=31
O resultado na base de treino é:  0.8250

# Conclusão

Fazer a feature selection não necessariamente vai melhorar os resultados do seu modelo.
Isso vai depender da situação analisada.

Se eu tenho variáveis altamente correlacionadas, a feature selection pode ser interessante
O mesmo se aplica para datasets com muitas colunas


Como não é o nosso caso, o resultado dos nossos modelos caiu, sendo mais interessante manter o dataset inteiro, sem feature selection